In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, GPTNeoModel, IntervalStrategy



In [2]:
#go find the model path in your directory from training and put it here
modelName = "../results/checkpoint-11214"
baseModelName = "EleutherAI/gpt-neo-125M"
model = GPTNeoModel.from_pretrained(modelName)#.cuda() #currently trying to do it on cpu
tokenizer = AutoTokenizer.from_pretrained(baseModelName)

In [42]:
words = 'cf'
input_ids = tokenizer(words)['input_ids']
input_ids_tensor = torch.tensor([input_ids])#.to('cuda')
input_embeddings = model.wte(input_ids_tensor)

In [41]:
print(input_embeddings)

tensor([[ 6.5448e-01, -1.1296e+00,  2.2994e-01, -2.0341e-01, -6.3902e-01,
         -8.3256e-02,  2.6878e-01, -2.3877e-01,  7.7936e-01,  6.3869e-01,
          1.7148e-01,  1.2027e-01,  6.3560e-01,  7.9103e-01, -2.8418e-01,
          2.5374e-02,  2.7414e-02, -2.9214e-01,  1.6073e-01, -9.5805e-01,
          4.6556e-01,  1.3916e-01, -2.0575e-01,  2.6118e-01,  5.0232e-01,
          5.9292e-02, -2.1156e-01,  4.0561e-02,  1.0443e+00, -3.2141e-01,
         -1.2920e-01,  1.7640e-01,  3.0001e-01, -7.5592e-01, -6.4696e-02,
          8.0000e-02, -5.4271e-02,  7.2325e-02, -1.2082e-01,  5.5735e-01,
          3.3612e-02, -6.5067e-01,  8.0707e-03,  4.6758e-01, -2.6660e-01,
         -1.6951e-01, -1.8699e-01,  2.9610e-01, -4.5746e-01,  2.1595e-01,
         -3.2726e-01, -2.4433e-01, -2.1913e-02,  1.9859e-01,  1.4514e-01,
         -1.2462e-01, -6.2709e-01, -5.8199e-02, -5.0128e-02, -2.5907e-01,
         -6.2341e-01, -2.1525e-02, -4.6281e-03,  1.7830e-01,  9.5691e-02,
         -5.6900e-01, -2.2284e-01,  8.